In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
import nltk
from nltk.corpus import stopwords

from io import StringIO
from html.parser import HTMLParser
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import json
 
nltk.download('stopwords')
nltk_stopwords = set(stopwords.words('english'))
nltk_stopwords

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased',truncation_side='left',truncation=True)
tokenizer.add_tokens(list(open('latex-vocabulary/latex_symbols.txt','r')))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


13855

In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    x = torch.ones(1, device=device)
    print(x)
else:
    print("MPS device not found.")
    device = torch.device("cpu")

MPS device not found.


In [3]:
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def get_latex_from_alt(context):
    strip_deliminators = lambda latex: latex.replace('$','').replace('\\[','').replace('\\]','')

    context_soup = BeautifulSoup(context)
    latex_images = context_soup.find_all('img')
    for image in latex_images:
        image.replace_with(strip_deliminators(image['alt']))
    # return [strip_deliminators(image['alt']) for image in latex_images]
    return str(context_soup)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def remove_stopwords(text):
    filtered_text = [w for w in text.split() if w.lower() not in nltk_stopwords]
    return " ".join(filtered_text)

Opening the JSON file where all the problems are stored:

In [6]:
problems = json.load(open('amc_12_problems_with_sol.json'))
problems["2015 AMC 12A #2"]["problem"]

'<html><head></head><body><p>Two of the three sides of a triangle are 20 and 15. Which of the following numbers is not a possible perimeter of the triangle?\n</p><p>\n</p></body></html>'

We must filter for problems from before 2019 and get their problem, solutions, and choices.

The following functions need to be applied to the text to simplify them:

In [7]:
problem_text = strip_tags(get_latex_from_alt(problem))
solutions_text = strip_tags(get_latex_from_alt(" ".join([solution for solution in json.loads(solutions_list) if 'http' not in solution])))
choices_text = " ".join(json.loads(choices))
training_text = " ".join([problem_text, solutions_text, choices_text])

NameError: name 'problem' is not defined

This part introduces how to apply the model to a certain string of text.

In [8]:
class DistilBERTClass(torch.nn.Module):
    def __init__(self, num_classes):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        return self.classifier(pooler)

The model must be downloaded: https://huggingface.co/yuppyd/top-level-with-solutions-distilbert-amc12-2019-2022/tree/main

In [11]:
model = DistilBERTClass(num_classes=5)
model.to(device)
model = torch.load('top-level-with-solutions-distilbert-amc12-2019-2022.pt',map_location=device)

def outputs(input_string):
    inputs = tokenizer.encode_plus(
            input_string,
            None,
            add_special_tokens=True,
            max_length=256,
            pad_to_max_length=True,
            return_tensors="pt").to(device)
    fin_outputs = []
    with torch.no_grad():
        outputs = model(**inputs)
    fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    outputs = np.array(fin_outputs) >= 0.5
    return [[1 if value else 0 for value in output] for output in outputs][0]

testA = "AMC 12A"
testB = "AMC 12B"
output = "";
for year in range(2015,2022):
    for i in range(0,2):
        for problem_num in range(1,26):
            if i==0:
                problem_ind = str(year)+" "+testA+" #"+str(problem_num)
            elif i==1:
                problem_ind = str(year)+" "+testB+" #"+str(problem_num)
            try: 
                problem_text = strip_tags(get_latex_from_alt(problems[problem_ind]["problem"]))
                solutions_text = strip_tags(get_latex_from_alt(" ".join([solution for solution in problems[problem_ind]["solutions"] if 'http' not in solution])))
                choices_text = " ".join(problems[problem_ind]["choices"])
                combined_text = remove_stopwords(" ".join([problem_text, solutions_text, choices_text]))

                assignment = outputs(combined_text)
                if assignment==[0,0,0,0,0]:
                    assignment[0] = 1;
                output+=problem_ind+": "+str(assignment)+"\n"

                with open('outputs_file_12.txt', 'w') as f:
                    f.write(output)
            except:
                continue

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\Anna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Store the outputs of every problem with their corresponding problem number / information